In [1]:
# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'

import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

PATH = r'C:\Users\user\PycharmProjects\DeepLearningSchool\8.Algorithm_composition\{}'
data = pd.read_csv(PATH.format('train.csv'))
features_labels = []
print(f"Количество строк {data.shape[0]}, количество признаков {data.shape[1]}")

# Чтение данных для обучения модели
y = data['Churn']
data.drop(['Churn'], inplace=True, axis=True)
data['TotalSpent'] = data['TotalSpent'].replace(' ', '0')
data['TotalSpent'] = data['TotalSpent'].astype(float)
data['TotalSpent'] = data['TotalSpent'].replace(0, data['TotalSpent'].mean())

# Чтение данных для предсказания
data_predict = pd.read_csv(PATH.format('test.csv'))
data_predict['TotalSpent'] = data_predict['TotalSpent'].replace(' ', '0')
data_predict['TotalSpent'] = data_predict['TotalSpent'].astype(float)
data_predict['TotalSpent'] = data_predict['TotalSpent'].replace(0, data['TotalSpent'].mean())
X_predict = data_predict

#LabelEncoder для категориальных признаков
le = LabelEncoder()
for s in cat_cols:
    le.fit(data[s])
    data[s+'_le'] = le.transform(data[s])
    X_predict[s+'_le'] = le.transform(X_predict[s])
    features_labels.append(s+'_le')

# OneHotEncoder для категориальных признаков
ohe = OneHotEncoder(handle_unknown='ignore')
for s in cat_cols:
    ohe.fit(data[s].values.reshape(-1, 1))
    
    new_ohe_features = ohe.transform(X_predict[s].values.reshape(-1, 1)).toarray()
    tmp = pd.DataFrame(new_ohe_features, columns=[s+'='+str(i) for i in range(new_ohe_features.shape[1])])
    X_predict = pd.concat([X_predict, tmp], axis=1)
    X_predict.drop([s], axis=1, inplace=True)
    
    new_ohe_features = ohe.transform(data[s].values.reshape(-1, 1)).toarray()
    tmp = pd.DataFrame(new_ohe_features, columns=[s+'='+str(i) for i in range(new_ohe_features.shape[1])])
    data = pd.concat([data, tmp], axis=1)
    data.drop([s], axis=1, inplace=True)
    
    [features_labels.append(s+'='+str(i)) for i in range(new_ohe_features.shape[1])]

# Нормализация числовых признаков
scaler = StandardScaler()
for i in num_cols:
    scaler.fit(data[i].values.reshape(-1, 1))
    
    X_predict[i+'_ss'] = scaler.transform(X_predict[i].values.reshape(-1, 1))
    X_predict.drop([i], axis=1, inplace=True)
    
    data[i+'_ss'] = scaler.transform(data[i].values.reshape(-1, 1))
    data.drop([i], axis=1, inplace=True)
    
    features_labels.append(i+'_ss') 

# Разбивание выборки для обучения на тестовую и валидационную
X_train, X_valid, y_train, y_valid = train_test_split(
    data, y, stratify=y, random_state=42, test_size=0.2
)

Количество строк 5282, количество признаков 20


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score



##############################################################################################
# GradientBoostingClassifier
parameters_gbc = {
        'loss': ['exponential'],
        'learning_rate': [0.05],
        'n_estimators': [100],
        'criterion': ['friedman_mse']
        }
X_train_xgb, X_valid_xgb, y_train_xgb, y_valid_xgb = train_test_split(
    data, y, stratify=y, random_state=42, test_size=0.2
)
clf_gbc = GradientBoostingClassifier()
gbc_grid = GridSearchCV(clf_gbc, parameters_gbc, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
gbc_grid.fit(X_train, y_train)
##############################################################################################


##############################################################################################
# MLPClassifier
parameters_mlpc = {
    'activation': ['tanh'],
    'solver': ['lbfgs'],
    'max_iter': (100, 500, 1000),
    'hidden_layer_sizes': [1]
}
clf_mlpc = MLPClassifier()
mlpc_grid = GridSearchCV(clf_mlpc, parameters_mlpc, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
mlpc_grid.fit(X_train, y_train)

##############################################################################################


##############################################################################################
# LogisticRegression
# l1-регуляризация говорит о том, что не все признаки полезны
parameters_lr= {
    'penalty': ['l1'],
    'C': [10],
    'class_weight': [None],
    'max_iter': [1000],
    'solver': ['saga']
}
clf_lr = LogisticRegression()
lr_grid = GridSearchCV(clf_lr, parameters_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
lr_grid.fit(X_train, y_train)
##############################################################################################


##############################################################################################
# GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(X_train, y_train)
##############################################################################################


##############################################################################################
# RandomForestClassifier
parameters_rfc = {
    'n_estimators': [150],
    'criterion': ['entropy'],
    'min_samples_leaf': [20],
    'max_features': ['sqrt'],
    'bootstrap':[False],
    'max_samples': [10]
}

clf_rfc = RandomForestClassifier()
rfc_grid = GridSearchCV(clf_rfc, parameters_rfc, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
rfc_grid.fit(X_train, y_train)
##############################################################################################


##############################################################################################
estimators = [
              ('gbc_grid', gbc_grid.best_estimator_),
              ('mlpc_grid', mlpc_grid.best_estimator_),
              ('lr_grid', lr_grid.best_estimator_),
              ('clf_gnb', clf_gnb),
              ('rfc_grid', rfc_grid.best_estimator_)
]

sc_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)
parameters_sc_lr = {
    'final_estimator__penalty': ['l2'],
    'final_estimator__solver': ['liblinear'],
    'final_estimator__class_weight': [None],
    'final_estimator__C': [1],
    'final_estimator__max_iter': [2000],
    'stack_method': ['predict_proba']
}
grid_sc_sklearn_clf = GridSearchCV(sc_clf, parameters_sc_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_sc_sklearn_clf.fit(X_train, y_train)
print('MLPClassifier - best estimator: ', grid_sc_sklearn_clf.best_estimator_)
print("Предсказание класса через лучшие параметры параметры", 
      roc_auc_score(y_valid, grid_sc_sklearn_clf.best_estimator_.predict_proba(X_valid)[:, 1]))
##############################################################################################

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.9s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:    0.6s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:    1.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.3s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.5s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   23.3s remaining:   35.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.7s finished
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


MLPClassifier - best estimator:  StackingClassifier(cv=5,
                   estimators=[('gbc_grid',
                                GradientBoostingClassifier(ccp_alpha=0.0,
                                                           criterion='friedman_mse',
                                                           init=None,
                                                           learning_rate=0.05,
                                                           loss='exponential',
                                                           max_depth=3,
                                                           max_features=None,
                                                           max_leaf_nodes=None,
                                                           min_impurity_decrease=0.0,
                                                           min_impurity_split=None,
                                                           min_samples_leaf=1,
                                

In [4]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score


#############################################################################################
params_xgb = {
        'min_child_weight': [5],
        'gamma': [5],
        'subsample': [0.1],
        'colsample_bytree': [0.5, 0.55, 0.6, ],
        'max_depth': [2]
        }

clf_xgb = XGBClassifier()
xgb_grid = GridSearchCV(clf_xgb, params_xgb, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
xgb_grid.fit(X_train, y_train)
#############################################################################################


#############################################################################################
parametrs_lgb = {
    'num_leaves': [6],
   'max_depth': [4],
   'class_weight': ['balanced'], 
   'random_state': [100],
   'learning_rate': [0.05],
   'n_estimators': [110]
}

clf_lgb = lgb.LGBMClassifier()
lgb_grid = GridSearchCV(clf_lgb, parametrs_lgb, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
lgb_grid.fit(X_train, y_train)
#############################################################################################


#############################################################################################
parameters_lr= {
    'penalty': ['elasticnet', 'l1', 'l2', 'none'],
    'C': [0.01, 0.1, 1, 10, 20, 50],
    'class_weight': [None],
    'max_iter': [1000],
    'solver': ['saga']
}
clf_lr = LogisticRegression()
lr_grid = GridSearchCV(clf_lr, parameters_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
lr_grid.fit(X_train, y_train)
#############################################################################################


#############################################################################################
cbc_clf = CatBoostClassifier(iterations=400,
                       depth=4,
                       learning_rate=0.05,
                       loss_function='Logloss',
                       verbose=False,
                       random_seed = 4,
                       l2_leaf_reg = 40,
                       eval_metric='AUC'
                       )

parametrs_cbc = {
    'iterations': [160],
    "learning_rate": [0.05],
    'min_data_in_leaf': [6],
    'depth': [5],
    'l2_leaf_reg': [0]    
}

cbc_grid = GridSearchCV(cbc_clf, parametrs_cbc, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
cbc_grid.fit(X_train, y_train)
#############################################################################################

estimators = [
     ('xgb', xgb_grid.best_estimator_),
     ('lgb', lgb_grid.best_estimator_),
     ('cbc', cbc_grid.best_estimator_),
    ('lr', lr_grid.best_estimator_)
]

sc_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)
parameters_sc_lr = {
    'final_estimator__C': [10, 1, 0.1],
    'final_estimator__penalty': ['elasticnet'],
    'final_estimator__solver': ['saga'],
    'final_estimator__l1_ratio': [0.5, 0.9],
    'stack_method': ['predict_proba']
}
gs_sc_estim_clf = GridSearchCV(sc_clf, parameters_sc_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_sc_estim_clf.fit(X_train, y_train)
print('StackingClassifier - best estimator: ', gs_sc_estim_clf.best_estimator_)
print("Предсказание вероятности ", roc_auc_score(y_valid, gs_sc_estim_clf.best_estimator_.predict_proba(X_valid)[:, 1]))

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:    0.2s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.5s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  97 out of 120 | elapsed:   13.9s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   18.9s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.6s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  30 | elapsed:   50.9s remaining:   33.9s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.1min finished


StackingClassifier - best estimator:  StackingClassifier(cv=5,
                   estimators=[('xgb',
                                XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=0.5, gamma=5,
                                              learning_rate=0.1,
                                              max_delta_step=0, max_depth=2,
                                              min_child_weight=5, missing=None,
                                              n_estimators=100, n_jobs=1,
                                              nthread=None,
                                              objective='binary:logistic',
                                              random_state=0, reg_alpha=0,
                                              reg_lambda=1, scale_p...
                               

In [7]:
from sklearn.ensemble import VotingClassifier
vk_clf = VotingClassifier(
    estimators= [
        ('grid_sc_sklearn_clf', grid_sc_sklearn_clf.best_estimator_),
        ('xgb', xgb_grid.best_estimator_),
        ('lgb', lgb_grid.best_estimator_),
        ('cbc', cbc_grid.best_estimator_)
    ],
    voting='soft')

parameters_vk = {
    'n_jobs': [-1]
}
gs_vk_clf = GridSearchCV(estimator=vk_clf, param_grid=parameters_vk, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_vk_clf.fit(X_train, y_train)


# submission = pd.DataFrame(gs_vk_clf.best_estimator_.predict_proba(X_predict)[:, 1], columns=["Churn"])
# submission.to_csv(PATH.format('my_submission_voiting.csv'), columns=["Churn"])
print('VotingClassifier - best estimator: ', gs_vk_clf.best_estimator_)
print('VotingClassifier - best params: ', gs_vk_clf.best_params_)
print("Предсказание вероятности ", roc_auc_score(y_valid, gs_vk_clf.best_estimator_.predict_proba(X_valid)[:, 1]))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   26.3s remaining:   39.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   28.7s finished


VotingClassifier - best estimator:  VotingClassifier(estimators=[('grid_sc_sklearn_clf',
                              StackingClassifier(cv=5,
                                                 estimators=[('gbc_grid',
                                                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                         criterion='friedman_mse',
                                                                                         init=None,
                                                                                         learning_rate=0.05,
                                                                                         loss='exponential',
                                                                                         max_depth=3,
                                                                                         max_features=None,
                                   

In [6]:
submission = pd.DataFrame(gs_vk_clf.best_estimator_.predict_proba(X_predict)[:, 1], columns=["Churn"])
submission.to_csv(PATH.format('my_submission_voiting.csv'), columns=["Churn"])

In [ ]:
import time
import pandas as pd
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier

t = time.time()
PATH = r'C:\Users\user\PycharmProjects\DeepLearningSchool\8.Algorithm_composition\{}'

##############################################################################################
# from google.colab import drive
# drive.mount('/content/drive')
# Читаем данные
data = pd.read_csv(PATH.format('train.csv'))
print(f"Количество строк {data.shape[0]}, количество признаков {data.shape[1]}")
y = data['Churn']
data.drop(['Churn'], inplace=True, axis=True)
data['TotalSpent'] = data['TotalSpent'].replace(' ', '0')
data['TotalSpent'] = data['TotalSpent'].astype(float)
data['TotalSpent'] = data['TotalSpent'].replace(0, data['TotalSpent'].mean())
X_train, X_valid, y_train, y_valid = train_test_split(
    data, y, stratify=y, random_state=42, test_size=0.2
)
##############################################################################################


##############################################################################################
data_predict = pd.read_csv(PATH.format('test.csv'))
data_predict['TotalSpent'] = data_predict['TotalSpent'].replace(' ', '0')
data_predict['TotalSpent'] = data_predict['TotalSpent'].astype(float)
data_predict['TotalSpent'] = data_predict['TotalSpent'].replace(0, data['TotalSpent'].mean())
X_predict = data_predict.copy()
##############################################################################################


##############################################################################################
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

features_labels = []
##############################################################################################


##############################################################################################
### LabelEncoder для категориальных признаков

le = LabelEncoder()
for s in cat_cols:
    le.fit(data[s])
    data[s+'_le'] = le.transform(data[s])
    X_predict[s+'_le'] = le.transform(X_predict[s])
    features_labels.append(s+'_le')
##############################################################################################


##############################################################################################
### OneHotEncoder для категориальных признаков
ohe = OneHotEncoder(handle_unknown='ignore')
for s in cat_cols:
    ohe.fit(data[s].values.reshape(-1, 1))
    
    new_ohe_features = ohe.transform(X_predict[s].values.reshape(-1, 1)).toarray()
    tmp = pd.DataFrame(new_ohe_features, columns=[s+'='+str(i) for i in range(new_ohe_features.shape[1])])
    X_predict = pd.concat([X_predict, tmp], axis=1)
    X_predict.drop([s], axis=1, inplace=True)
    
    new_ohe_features = ohe.transform(data[s].values.reshape(-1, 1)).toarray()
    tmp = pd.DataFrame(new_ohe_features, columns=[s+'='+str(i) for i in range(new_ohe_features.shape[1])])
    data = pd.concat([data, tmp], axis=1)
    data.drop([s], axis=1, inplace=True)
    
    [features_labels.append(s+'='+str(i)) for i in range(new_ohe_features.shape[1])]
##############################################################################################


##############################################################################################
### Нормализация числовых признаков
scaler = StandardScaler()
for i in num_cols:
    scaler.fit(data[i].values.reshape(-1, 1))
    
    X_predict[i+'_ss'] = scaler.transform(X_predict[i].values.reshape(-1, 1))
    X_predict.drop([i], axis=1, inplace=True)
    
    data[i+'_ss'] = scaler.transform(data[i].values.reshape(-1, 1))
    data.drop([i], axis=1, inplace=True)
    
    features_labels.append(i+'_ss') 
##############################################################################################

##############################################################################################
cbc = CatBoostClassifier(iterations=400,
                       depth=4,
                       learning_rate=0.05,
                       loss_function='Logloss',
                       verbose=False,
                       cat_features=cat_cols,
                       random_seed = 4,
                       l2_leaf_reg = 40,
                       eval_metric='AUC'
                       )

parametrs_cbc = {
    'iterations': [160, ],
    "learning_rate": [0.05, ],
    'min_data_in_leaf': [6,],
    'depth': [5,],
    'l2_leaf_reg': [0, ]    
}

cb_grid = GridSearchCV(cbc, parametrs_cbc, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
cb_grid.fit(X_train, y_train)
# print("CatBoostClassifier")
# print("Предсказание класса ", roc_auc_score(y_valid, cb_grid.best_estimator_.predict(X_valid)))
# print("Предсказание класса через лучшие параметры параметры", roc_auc_score(y_valid, cb_grid.best_estimator_.predict(X_valid)))
# print("Предсказание вероятности ", roc_auc_score(y_valid, cb_grid.predict_proba(X_valid)[:, 1]))
# print("Лучшие параметры ", cb_grid.best_params_)

# submission = pd.DataFrame(cb_grid.best_estimator_.predict_proba(data_predict)[:, 1], columns=["Churn"])
# submission.to_csv(PATH.format('my_submission_catboost.csv'), columns=["Churn"])
##############################################################################################


##############################################################################################
parametrs_lgb = {
    'num_leaves': [5],
   'max_depth': [4],
   'class_weight': ['balanced'], 
   'learning_rate': [0.05],
    'subsample_for_bin': [200],
   'n_estimators': [115]
}


X_train_lgb, X_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(
    data, y, stratify=y, random_state=42, test_size=0.2
)
clf_lgb = lgb.LGBMClassifier()
lgb_grid = GridSearchCV(clf_lgb, parametrs_lgb, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
lgb_grid.fit(X_train_lgb, y_train_lgb)
# print("LGBMClassifier")
# print("Предсказание класса через лучшие параметры параметры", 
#       roc_auc_score(y_valid_lgb, lgb_grid.best_estimator_.predict(X_valid_lgb)))

# submission = pd.DataFrame(lgb_grid.best_estimator_.predict_proba(X_predict)[:, 1], columns=["Churn"])
# submission.to_csv(PATH.format('my_submission_lgbm.csv'), columns=["Churn"])
##############################################################################################


##############################################################################################
parameters_mlpc = {
    'activation': ['identity'],
    'solver': ['lbfgs'],
    'max_iter': [110],
    'hidden_layer_sizes': [5]
}

mlpc_clf = MLPClassifier()
gs_mlpc_clf = GridSearchCV(mlpc_clf, parameters_mlpc, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_mlpc_clf.fit(X_train_lgb, y_train_lgb)
# print('StackingClassifier - best estimator: ', gs_mlpc_clf.best_estimator_)
# print("MLPClassifier - best predict: ", roc_auc_score(y_valid_xgb, gs_mlpc_clf.predict_proba(X_valid_xgb)[:, 1]))
##############################################################################################


##############################################################################################
params_xgb = {
        'min_child_weight': [5],
        'gamma': [5],
        'subsample': [0.1],
        'colsample_bytree': [0.5, 0.55, 0.6],
        'max_depth': [2]
        }
X_train_xgb, X_valid_xgb, y_train_xgb, y_valid_xgb = train_test_split(
    data, y, stratify=y, random_state=42, test_size=0.2
)
clf_xgb = XGBClassifier()
xgb_grid = GridSearchCV(clf_xgb, params_xgb, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
xgb_grid.fit(X_train_xgb, y_train_xgb)
# print("XGBClassifier")
# print("Предсказание класса через лучшие параметры параметры", roc_auc_score(y_valid_xgb, xgb_grid.best_estimator_.predict(X_valid_xgb)))
# print("Предсказание вероятности ", roc_auc_score(y_valid_xgb, xgb_grid.best_estimator_.predict_proba(X_valid_xgb)[:, 1]))
# print("Лучшие параметры ", xgb_grid.best_params_)

# submission = pd.DataFrame(xgb_grid.best_estimator_.predict_proba(X_predict)[:, 1], columns=["Churn"])
# submission.to_csv(PATH.format('my_submission_xgb.csv'), columns=["Churn"])
##############################################################################################


##############################################################################################
estimators = [
     ('xgb', xgb_grid.best_estimator_),
     ('lgb', lgb_grid.best_estimator_),
     ('mplpc', gs_mlpc_clf.best_estimator_)
#      ('cb', cb_grid.best_estimator_)
]


sc_clf = StackingClassifier(estimators=estimators, final_estimator=MLPClassifier(), cv=5)
parameters_sc_lr = {
    'final_estimator__activation': ['logistic'],
    'final_estimator__solver': ['lbfgs'],
    'final_estimator__hidden_layer_sizes': [100],
    'stack_method': ['predict_proba']
}
gs_sc_clf = GridSearchCV(sc_clf, parameters_sc_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_sc_clf.fit(X_train_lgb, y_train_lgb)

# End estimate. Writing some info 
submission = pd.DataFrame(gs_sc_clf.best_estimator_.predict_proba(X_predict)[:, 1], columns=["Churn"])
submission.to_csv(PATH.format('my_submission_stack.csv'), columns=["Churn"])
print('StackingClassifier - best estimator: ', gs_sc_clf.best_estimator_)
print('StackingClassifier - best params: ', gs_sc_clf.best_params_)
print("Предсказание вероятности ", roc_auc_score(y_valid_xgb, gs_sc_clf.best_estimator_.predict_proba(X_valid_xgb)[:, 1]))
##############################################################################################


##############################################################################################

vk_clf = VotingClassifier(
    estimators= [
        ('xgb', xgb_grid.best_estimator_),
#         ('lgb', lgb_grid.best_estimator_),
#         ('mplpc', gs_mlpc_clf.best_estimator_)
#         ('cb', cb_grid.best_estimator_)
    ],
    voting='soft')
parameters_vk = {
#     'estimator__voting': ['hard','soft'],
    'n_jobs': [-1]
}
gs_vk_clf = GridSearchCV(estimator=vk_clf, param_grid=parameters_vk, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_vk_clf.fit(X_train_lgb, y_train_lgb)


submission = pd.DataFrame(gs_vk_clf.best_estimator_.predict_proba(X_predict)[:, 1], columns=["Churn"])
submission.to_csv(PATH.format('my_submission_voiting.csv'), columns=["Churn"])
print('VotingClassifier - best estimator: ', gs_vk_clf.best_estimator_)
print('VotingClassifier - best params: ', gs_vk_clf.best_params_)
print("Предсказание вероятности ", roc_auc_score(y_valid_xgb, gs_vk_clf.best_estimator_.predict_proba(X_valid_xgb)[:, 1]))




##############################################################################################
##############################################################################################
##############################################################################################




from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

##############################################################################################
# LogisticRegression
parameters_lr= {
    'penalty': ['l1'],
    'C': [10],
    'class_weight': [None],
    'max_iter': [1000],
    'solver': ['saga']
}
clf_lr = LogisticRegression()
gs_lr_clf = GridSearchCV(clf_lr, parameters_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_lr_clf.fit(X_train_lgb, y_train_lgb)
print('LogisticRegression - best estimator: ', gs_sc_clf.best_estimator_)
print("Предсказание вероятности ", roc_auc_score(y_valid_lgb, gs_lr_clf.best_estimator_.predict_proba(X_valid_lgb)[:, 1]))
##############################################################################################


##############################################################################################
# GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(X_train_lgb, y_train_lgb)
print('GaussianNB')
print("Предсказание вероятности ", roc_auc_score(y_valid_lgb, clf_gnb.predict_proba(X_valid_lgb)[:, 1]))
##############################################################################################


##############################################################################################
# RandomForestClassifier
parameters_rfc = {
    'n_estimators': [10, 50, 100, 150, 200],
    'criterion': ['gini','entropy'],
    'min_samples_leaf': [1, 5, 10, 15, 20],
    'max_features': ['sqrt', 'log2'],
    'bootstrap':[False],
    'max_samples': [5, 10, 15, 20]
}

clf_rfc = RandomForestClassifier()
gs_rfc_clf = GridSearchCV(clf_lr, parameters_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_rfc_clf.fit(X_train_lgb, y_train_lgb)

print('RandomForestClassifier - best estimator: ', gs_sc_clf.best_estimator_)
print("Предсказание вероятности ", roc_auc_score(y_valid_lgb, gs_rfc_clf.best_estimator_.predict_proba(X_valid_lgb)[:, 1]))
##############################################################################################


estimators = [
     ('lr', gs_lr_clf.best_estimator_),
    ('xgb', xgb_grid.best_estimator_),
     ('lgb', lgb_grid.best_estimator_),
#      ('gnb', clf_gnb),
     ('rfc', gs_rfc_clf.best_estimator_)
]


sc_clf = StackingClassifier(estimators=estimators, final_estimator=MLPClassifier(), cv=5)
parameters_sc_lr = {
#     'final_estimator__activation': ['logistic'],
#     'final_estimator__solver': ['lbfgs'],
#     'final_estimator__hidden_layer_sizes': [100],
    'stack_method': ['predict_proba']
}
gs_sc_clf = GridSearchCV(sc_clf, parameters_sc_lr, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_sc_clf.fit(X_train_lgb, y_train_lgb)
submission = pd.DataFrame(gs_sc_clf.best_estimator_.predict_proba(X_predict)[:, 1], columns=["Churn"])
submission.to_csv(PATH.format('my_submission_stack.csv'), columns=["Churn"])
print('StackingClassifier - best estimator: ', gs_sc_clf.best_estimator_)
print('StackingClassifier - best params: ', gs_sc_clf.best_params_)
print("Предсказание вероятности ", roc_auc_score(y_valid_xgb, gs_sc_clf.best_estimator_.predict_proba(X_valid_xgb)[:, 1]))
print(f'How long: {"%.2f" % (time.time()-t)}, s')

In [ ]:
##############################################################################################
# RandomForestClassifier
parameters_rfc = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 300],
    'criterion': ['entropy'],
    'min_samples_leaf': [20],
    'max_features': ['sqrt'],
    'bootstrap':[False],
    'max_samples': [10]
}

clf_rfc = RandomForestClassifier()
gs_rfc_clf = GridSearchCV(clf_rfc, parameters_rfc, cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)
gs_rfc_clf.fit(X_train_lgb, y_train_lgb)

print('RandomForestClassifier - best estimator: ', gs_rfc_clf.best_estimator_)
print("Предсказание вероятности ", roc_auc_score(y_valid_lgb, gs_rfc_clf.best_estimator_.predict_proba(X_valid_lgb)[:, 1]))
##############################################################################################

In [ ]:
X_predict.head()

cbc = CatBoostClassifier(iterations=400,
                       depth=4,
                       learning_rate=0.05,
                       loss_function='Logloss',
                       verbose=False,
                       random_seed = 4,
                       l2_leaf_reg = 40,
                       eval_metric='AUC'
                       )

parametrs_cbc = {
    'iterations': [120, 150, 160],
    "learning_rate": [0.03],
    'min_data_in_leaf': [2],
    'depth': [4],
    'l2_leaf_reg': [0.1]    
}

cb_grid = GridSearchCV(cbc, parametrs_cbc, cv=5, verbose=4, scoring='roc_auc', refit=True, n_jobs=-1)
cb_grid.fit(X=X_train_lgb, y=y_train_lgb)
print("CatBoostClassifier")
print('StackingClassifier - best estimator: ', cb_grid.best_params_)
print("Предсказание класса через лучшие параметры параметры", roc_auc_score(y_valid_lgb, cb_grid.best_estimator_.predict(X_valid_lgb)))
print("Предсказание вероятности ", roc_auc_score(y_valid, cb_grid.predict_proba(X_valid_lgb)[:, 1]))